In [1]:
import os
from datetime import datetime
import pydoop.hdfs as hdfs
import re
import pandas as pd
import pyarrow as pa
import sys

In [2]:
dirs = os.listdir('/home/itversity/itversity-material/Retail_pipeline_project/data')
groups = [d for d in dirs if os.path.isdir(os.path.join('/home/itversity/itversity-material/Retail_pipeline_project/data', d)) and d.startswith('group')]
print(groups)

['group1', 'group2', 'group3', 'group4', 'group5', 'group6']


- Function to write ingested files into a text file to skip them on the next round

In [3]:
def write_dirs_to_file(file_path, group):
    with open(file_path, 'r') as file:
            existing_content = file.read().splitlines()
            
    with open(file_path, 'a') as file:
        if group not in existing_content:
            file.write(f"{group}\n")

In [4]:
with open('/home/itversity/itversity-material/Retail_pipeline_project/data/written_groups.txt', 'r') as file:
    skip_dir = file.read().splitlines()
    
filtered_dir = [group for group in groups if group not in skip_dir]

 Function to rename the files

In [5]:
def rename_files_and_upload(local_directory, hdfs_directory):
    files = os.listdir(local_directory)
    print(files)
    # Rename files
    for file_name in files:
        old_file_path = os.path.join(local_directory, file_name)
        
        # Get the current date and hour
        now = datetime.now()
        date_str = now.strftime("%Y%m%d")
        hour_str = now.strftime("%H")
        
        # Split the file name and extension
        split_name= os.path.splitext(file_name)
        # Create the new file name with the extension at the end
        print(split_name)
        new_parquet_file_name = f"{split_name[0][:-2]}_{date_str}_{hour_str}.parquet"
        new_parquet_file_path = os.path.join(local_directory, new_parquet_file_name)

        try:
            df = pd.read_csv(old_file_path, header = 0)
            prq = df.to_parquet(new_parquet_file_path)
        except:
            print("files already converted and ranamed")
            
        # Upload the renamed file to HDFS
        hdfs.mkdir(f"/data/retail_bronze/{date_str}/{hour_str}")
        day_directory = os.path.join(hdfs_directory, date_str)
        hour_directory = os.path.join(day_directory, hour_str)
        hdfs_file_path = os.path.join(hour_directory, new_parquet_file_name)
        hdfs.put(new_parquet_file_path, hdfs_file_path)
        print(f"Uploaded {new_parquet_file_name} to {hdfs_file_path}")


In [6]:
try:
    if filtered_dir:
        rename_files_and_upload(f"/home/itversity/itversity-material/Retail_pipeline_project/data/{filtered_dir[0]}", '/data/retail_bronze')
    else:
        raise SystemExit(f"No new groups found")  # Exit with code 1 for missing path
except Exception as e:
    print(f"An error occurred: {e}")
    raise SystemExit(1)  # Exit with code 1 for other errors

['branches_SS_raw_1.csv', 'sales_agents_SS_raw_1.csv', 'sales_transactions_SS_raw_1.csv']
('branches_SS_raw_1', '.csv')
Uploaded branches_SS_raw_20240713_07.parquet to /data/retail_bronze/20240713/07/branches_SS_raw_20240713_07.parquet
('sales_agents_SS_raw_1', '.csv')
Uploaded sales_agents_SS_raw_20240713_07.parquet to /data/retail_bronze/20240713/07/sales_agents_SS_raw_20240713_07.parquet
('sales_transactions_SS_raw_1', '.csv')
Uploaded sales_transactions_SS_raw_20240713_07.parquet to /data/retail_bronze/20240713/07/sales_transactions_SS_raw_20240713_07.parquet


write the already ingested files to the log

In [7]:
write_dirs_to_file('/home/itversity/itversity-material/Retail_pipeline_project/data/written_groups.txt', filtered_dir[0])